In [1]:
#libraries
import os 
import sys

In [2]:
#first load modules then append
!module load intel/2017.4.056 amber/16
!export PATH=$HPC_AMBER_DIR/AmberTools/bin:$PATH

#next prepare your pdbfile, should be in your 
#directory to make life easy
!pdb4amber -i 2h24.pdb -o pIL10.pdb --reduce --dry 

/bin/bash: pdb4amber: command not found


In [ ]:
#Next we'll use leap to prepare teh topology file
%%file leap.in
source leaprc.protein.ff14SB
source leaprc.water.tip3p
source leaprc.gaff2
x = loadPdb new.pdb
saveAmberParm x my_new_pdb.parm7 my_new_pdb.rst7
quit

In [ ]:
#activate tleap
!tleap -f leap.in > leap.log

In [ ]:
#TO DO AMBER, YOU NEED TO DO A SERIES OF SIMULATIONS TO GET THE PRODUCTION VERSION OF YOUR STRUCTURE THEN RUN FINAL SIMULATION

#STEP 1 MINIMIZATION
%%file min.in
  simple generalized Born minimization script
 &cntrl
   imin=1, ntb=0, maxcyc=100, ntpr=10, cut=1000., igb=2, 
 /

In [ ]:
#MINIMIZE
sander -O -i min.in -p my_new_pdb.parm7 \
       -c my_new_pdb.rst7 \
       -o min1.out \
       -r min1.rst7

In [ ]:
#STEP 2 PRODUCTION RUN 
%%file md.in
Implicit solvent molecular dynamics
 &cntrl
   imin=0, irest=1, ntx=5,
   ntpr=1000, ntwx=1000, nstlim=500000,
   dt=0.002, ntt=3, tempi=300,
   temp0=300, gamma_ln=1.0, ig=-1,
   ntp=0, ntc=2, ntf=2, cut=1000,
   ntb=0, igb=2, ioutfm=1,
 /

In [ ]:
#MINIMIZE BUT ITS VERY IMPORTANT TO DO IT ON THE STRUCTURE YOU'VE MINIMIZED, SO THE SANDER BIT LOOKS LIKE THIS CAUSE YOU'll DO TWO SANDER RUNS:
sander -O -i md.in -p test.parm7 \ 
       -c test.rst7 -o md1.mdout \ 
       -x md1.nc -r md1.rst7 


In [16]:
# to chimera hooray. 

NGLWidget()